Задача: научиться находить в тексте лексически/стилистически неправильные сочетания (коллокации) и предлагать более правильные замены. <br>
Алгоритм является улучшенной версией этого алгоритма https://github.com/annadmitrieva/NLP-stuff/blob/master/collocation_generation-Copy4.ipynb 

#Этапы работы

*   Автоматическое обнаружение "неправильных" коллокаций в тексте.
*   Поиск замен.

In [0]:
from IPython.display import clear_output

In [0]:
!pip install natasha
!pip install treetaggerwrapper
clear_output()

Скачиваем word2vec модель из rusvectores.

In [0]:
!wget https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz
clear_output()

Для определения домена текста используем предобученную на шестиграммах из каждого домена (по 100000 на домен) SVM. F1-score модели – 0.94. <br>
Код модели: https://github.com/annadmitrieva/NLP-stuff/blob/master/domain_model.ipynb

In [0]:
!wget https://www.dropbox.com/s/lyfk0ur0pqvcgqv/domain_model.pkl?dl=0 -O domain_model.pkl
clear_output()

In [0]:
!wget https://www.dropbox.com/s/y35dm5chuv54psg/suggestions.zip?dl=0 -O suggestions.zip
clear_output()

In [6]:
!unzip suggestions.zip

Archive:  suggestions.zip
  inflating: suggestions_law.csv     
   creating: __MACOSX/
  inflating: __MACOSX/._suggestions_law.csv  
  inflating: suggestions_ling.csv    
  inflating: __MACOSX/._suggestions_ling.csv  
  inflating: suggestions_pol.csv     
  inflating: __MACOSX/._suggestions_pol.csv  
  inflating: suggestions_hist.csv    
  inflating: __MACOSX/._suggestions_hist.csv  
  inflating: suggestions_psy.csv     
  inflating: __MACOSX/._suggestions_psy.csv  
  inflating: suggestions_soc.csv     
  inflating: __MACOSX/._suggestions_soc.csv  
  inflating: suggestions_ec.csv      
  inflating: __MACOSX/._suggestions_ec.csv  


In [0]:
import pandas as pd
import re
import nltk
import gensim
from gensim.models import Word2Vec
from treetaggerwrapper import TreeTagger
from natasha import NamesExtractor, AddressExtractor, DatesExtractor
from sklearn.externals import joblib

model = gensim.models.KeyedVectors.load_word2vec_format('ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz', binary=False)
domain_model=joblib.load('domain_model.pkl')

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
def domain(string):
    domain=domain_model.predict(list(string))[0]
    df=pd.read_csv(f'suggestions_{domain}.csv')
    return df

#Поиск неправильных сочетаний

Для начала в тексте находятся все пары "вершина-зависимое", которые состоят из существительных и глаголов и которых нет в эталонных списках правильных словосочетаний (для каждого домена текста – свой список).<br>
Для морфологической разметки использовалась питоновская обёртка для TreeTagger.<br>
Для синтаксического парсинга использовался MaltParser из NLTK с предобученной моделью для русского языка. <br>
Источник модели: http://corpus.leeds.ac.uk/mocky/ <br>
Эталонные списки коллокаций: https://drive.google.com/drive/folders/1k_N-DZ-nLL5ro66-LxIaE4-dRwirdwZh <br>
Код, которых использовался при сборе эталонных списков: https://github.com/MariaFjodorowa/catandthekittens/tree/develop/collocations/collocation_frequencies

In [0]:
!wget http://corpus.leeds.ac.uk/tools/russian.mco
!wget http://maltparser.org/dist/maltparser-1.8.1.zip
!unzip maltparser-1.8.1.zip
clear_output()

In [0]:
!export MALT_PARSER=$HOME/maltparser-1.8.1/
!export MALT_MODEL=$HOME/russian.mco

In [0]:
syntax_parser = nltk.parse.malt.MaltParser('maltparser-1.8.1', 'russian.mco')

Несколько плохих текстов, на которых будем проверять, насколько хорошо находятся плохие коллокации. Источник – примеры плохой лексической сочетаемости из домашнего задания по академическому письму 1 курса ОП "Фундаментальная и прикладная лингвистика" НИУ ВШЭ 2017 года.

In [0]:
bad_text_pol = """В то же время, Путин умолчал от египетского населения данные о том, 
насколько милитаризируется сама Россия и насколько выросли ее военные расходы. 
Обама улетел из Вашингтона на вертолете, а Байден - на поезде."""
bad_text_hist = """Маккензи ограничивается рамками исследования 1928 – 1943, исследуя преимущественно идеологию Коминтерна сталинской эпохи, 
перечисляет причины на то, чтобы рассматривать 1928 год как переломный. 
Александр Шубин посвящает свою книгу большей частью на исследование общественно-политических движений в послесталинский СССР, 
1953 – 85 годов. В строках читаются напряжение и недоверие власти."""
bad_text_ling = """Следует избегать наводящие вопросы. Тем не менее существовало несколько точек зрения о создании алфавита. 
Предлагая ту или иную гипотезу, нельзя основываться и ссылаться только на результаты своего собственного исследования. 
Необходимо внимательное изучение опыта других ученых. 
Термин морфология обычно соотносится к части грамматики языка. Однако этот термин в истории науки употреблялся и в совсем ином значении."""

In [20]:
#проверим, правильно ли работает парсер
sents = nltk.sent_tokenize(bad_text_pol)
nodes = syntax_parser.parse_one(sents[0].split()).nodes
for node in nodes.keys():
    head = nodes[node]['head']
    if head is not None and nodes[head]['word'] is not None:
        print(f"Head: {nodes[head]['word']} {nodes[head]['ctag']}, dependent: {nodes[node]['word']} {nodes[node]['ctag']}.")

Head: то NN, dependent: В NN.
Head: то NN, dependent: же NN.
Head: Путин NN, dependent: время, NN.
Head: умолчал NN, dependent: Путин NN.
Head: от NN, dependent: умолчал NN.
Head: то NN, dependent: от NN.
Head: от NN, dependent: египетского NN.
Head: египетского NN, dependent: населения NN.
Head: о NN, dependent: данные NN.
Head: том, NN, dependent: о NN.
Head: насколько NN, dependent: том, NN.
Head: милитаризируется NN, dependent: насколько NN.
Head: сама NN, dependent: милитаризируется NN.
Head: Россия NN, dependent: сама NN.
Head: и NN, dependent: Россия NN.
Head: насколько NN, dependent: и NN.
Head: выросли NN, dependent: насколько NN.
Head: ее NN, dependent: выросли NN.
Head: военные NN, dependent: ее NN.
Head: то NN, dependent: военные NN.
Head: военные NN, dependent: расходы. NN.


/usr/local/lib/python3.6/dist-packages/nltk/parse/dependencygraph.py:380: UserWarning: The graph doesn't contain a node that depends on the root element.
  "The graph doesn't contain a node "


In [0]:
def search(text, collocations):
    clean_text = re.sub("\([a-zA-z\d .,:]*\)|[a-zA-z\d]*", "", text)
    #чистим текст от библиографических ссылок и токенов, состоящих только из латиницы и цифр
    sents = nltk.sent_tokenize(clean_text)
    #разбиваем на предложения
    for parse in syntax_parser.parse_sents(sents):